In [3]:
from pathlib import Path
import tabula
import pandas as pd
from PyPDF2 import PdfFileReader
from sqlalchemy import create_engine
pd.options.mode.chained_assignment = None

In [72]:
file = "D:/dvrpc_shared/BFR_ComparePavingPlans/data/PDFs/Chester County Five Year Plan 2022-2026.pdf"
print(file)

D:/dvrpc_shared/BFR_ComparePavingPlans/data/PDFs/Chester County Five Year Plan 2022-2026.pdf


In [73]:

def get_number_of_pages_in_pdf(filepath):
    """
    Open a PDF file and return the number of pages within
    """
    with open(filepath, "rb") as pdf_file:
        pdf_reader = PdfFileReader(pdf_file)
        number_of_pages = pdf_reader.numPages

    return number_of_pages

In [74]:
def fill_empty_years(df):
    frames = []
    r = df.index[df['SR']=='Result']
    c = df.index[df['Year']=='Calendar year']

    for i in range(0, len(r)):
        page_start = 0
        if i == 0:
            chunk = df.iloc[page_start:r[i]]
            #starting from after the misplaced heading name
            year = df.iloc[c[0]+1]['Year']
            chunk['Year'].fillna(year, inplace=True)
            frames.append(chunk)
        else:
            chunk = df.iloc[r[i-1]+1:r[i]]
            year = df.iloc[r[i-1]+1]['Year']
            chunk['Year'].fillna(year, inplace=True)
            frames.append(chunk)

    allyears = pd.concat(frames, ignore_index=True)
    
    return allyears

In [75]:
def drop_total_results_row(df):
    for idx, row in df.iterrows():
        if row['Year'] =="Result":
            df.drop(idx, inplace=True)
        elif row
    return df

In [153]:
df['Year','Municipality3'].head()

SyntaxError: invalid syntax (Temp/ipykernel_19948/557549271.py, line 1)

In [80]:
def parse_single_page(file, page_number):

    table = tabula.read_pdf(file, pages=page_number, pandas_options={'header': None})[0]
    df = pd.DataFrame(table)

    #drop extra/empty columns (empty except header row)
    cols_to_drop = []
    for i in range(0, len(df.columns)):
        if df[i].isnull()[1:].all():
            cols_to_drop.append(i)

    if len(cols_to_drop) > 0:
        df.drop(cols_to_drop, axis=1, inplace=True)

    #rename columns
    column_names = ["Year", "SR", "Road Name", "From", "SegmentFrom", "OffsetFrom", "To", "SegmentTo", "OffsetTo", "Municipality1", "Municipality2", "Municipality3", "Miles Panned"]
    df.columns = column_names

    #fill in empty years
    df = fill_empty_years(df)

    #check number of rows header is read into (differs by file)
    if str(df['SR'][0]) == 'nan':
        rows_to_drop = [0,1]
        df.drop(rows_to_drop, inplace=True)

    else:
        rows_to_drop = [0]
        df.drop(rows_to_drop, inplace=True)

    #drop the last row with the plan total
    df = drop_total_results_row(df)

    return df




In [165]:
file = "D:/dvrpc_shared/BFR_ComparePavingPlans/data/PDFs/Delaware County Five Year Plan 2022-2026.pdf"
table = tabula.read_pdf(file, pages=2, pandas_options={'header': None})[0]
df = pd.DataFrame(table)

cols_to_drop = []
for i in range(0, len(df.columns)):
    if df[i].isnull()[1:].all():
        cols_to_drop.append(i)

if len(cols_to_drop) > 0:
    df.drop(cols_to_drop, axis=1, inplace=True)

column_names = ["Year", "SR", "Road Name", "From", "SegmentFrom", "OffsetFrom", "To", "SegmentTo", "OffsetTo", "Municipality1", "Municipality2", "Municipality3", "Miles Panned"]
df.columns = column_names


In [166]:
df.index[df['SR'].isnull()]

Int64Index([17, 18], dtype='int64')

In [167]:
frames = []
r = df.index[df['SR']=='Result']
n = df.index[df['SR'].isnull()]
c = df.index[df['Year']=='Calendar year']

print(r, n, c)
#if there is no total result row because they were left null, the nulls will be next to each other
#drop the second one from the list
if len(r) == 0:
    print("No r")
    if len(n) > 1:
        print("n is long enough")
        #if there is no total result row because they were left null, the nulls will be next to each other
        #drop the second one from the list
        if n[-1] - n[-2] == 1:
            n=n[:-1]
    r = n
    print(r)

Int64Index([], dtype='int64') Int64Index([17, 18], dtype='int64') Int64Index([0], dtype='int64')
No r
n is long enough
Int64Index([17], dtype='int64')


In [168]:
if len(r) == 0:
    if len(n) > 1:
        #if there is no total result row because they were left null, the nulls will be next to each other
        #drop the second one from the list
        if n[-1] - n[-2] == 1:
            n=n[:-1]
    r = n
    for i in range(0, len(r)):
        page_start = 0
        if i == 0:
            chunk = df.iloc[page_start:r[i]]
            #starting from after the misplaced heading name
            year = df.iloc[c[0]+1]['Year']
            chunk['Year'].fillna(year, inplace=True)
            frames.append(chunk)
        else:
            chunk = df.iloc[r[i-1]+1:r[i]]
            year = df.iloc[r[i-1]+1]['Year']
            chunk['Year'].fillna(year, inplace=True)
            frames.append(chunk)
else: 
    for i in range(0, len(r)):
        page_start = 0
        if i == 0:
            chunk = df.iloc[page_start:r[i]]
            #starting from after the misplaced heading name
            year = df.iloc[c[0]+1]['Year']
            chunk['Year'].fillna(year, inplace=True)
            frames.append(chunk)
        else:
            chunk = df.iloc[r[i-1]+1:r[i]]
            year = df.iloc[r[i-1]+1]['Year']
            chunk['Year'].fillna(year, inplace=True)
            frames.append(chunk)

allyears = pd.concat(frames, ignore_index=True)
allyears.head()

,Year,SR,Road Name,From,SegmentFrom,OffsetFrom,To,SegmentTo,OffsetTo,Municipality1,Municipality2,Municipality3,Miles Panned
0,Calendar year,State RouteLoc Road Name RMS,LOCATION FROM\rIntersection FromSegment FromOf...,LOCATION TO\rIntersection ToSegment ToOffset To,Municipality Name1,Municipality Name2,Municipality Name3,Miles\rPlanned,NaN,NaN,NaN,NaN,NaN
1,2026,0202,WILMINGTON PK,Baltimore Pk / SR 0001,0090,0000,Dilworthtown Rd / SR 4016 / Chester Co. Line,0116,2108.0,CONCORD,CHADDS FORD,NaN,1.66
2,2026,0202,WILMINGTON PK,Baltimore Pk / SR 0001,0091,0000,Dilworthtown Rd / SR 4016 / Chester Co. Line,0117,2372.0,CONCORD,CHADDS FORD,NaN,1.65
3,2026,0291,INDUSTRIAL HW,320' E of Eddystone Industrial Ct,0114,0000,Stevens Dr.,0180,3053.0,EDDYSTONE,TINICUM,RIDLEY,3.22
4,2026,0291,INDUSTRIAL HW,320' E of Eddystone Industrial Ct,0115,0000,Stevens Dr.,0181,3043.0,EDDYSTONE,TINICUM,RIDLEY,3.23


In [117]:
page1 = parse_single_page(file, 1)
#page2 = parse_single_page(file, 2)
#page2.head()

ValueError: No objects to concatenate

In [ ]:
def parse_all_pdfs(county):
    file = fr"D:/dvrpc_shared/BFR_ComparePavingPlans/data/PDFs/{county} County Five Year Plan 2022-2026.pdf"
    frames = []
    num_pages = get_number_of_pages_in_pdf(file)
    for i in range(1, num_pages+1):
        df = parse_single_page(file, page_number=i)
        frames.append(df)

    allpgs = pd.concat(frames, ignore_index=True)

    output_filepath = fr"D:/dvrpc_shared/BFR_ComparePavingPlans/data/CSVs/{county}_fiveyearplan_2022_2026.csv"

    allpgs.to_csv(output_filepath, index=False)
    allpgs.to_sql(f"{county}_County_Plan", ENGINE, if_exists="replace")


counties = ["Bucks", "Chester", "Delaware", "Montgomery", "Philadelphia"]

for county in counties:
    parse_all_pdfs(county)
    


In [49]:
test = fill_empty_years(df)
test.head()

2022
2023
2024
2025


,Year,SR,Road Name,From,SegmentFrom,OffsetFrom,To,SegmentTo,OffsetTo,Municipality1,Municipality2,Municipality3,Miles Panned
0,Calendar year,State RouteLoc Road Name RMS,LOCATION FROM\rIntersection FromSegment FromOf...,LOCATION TO\rIntersection ToSegment ToOffset To,Municipality Name1,Municipality Name2,Municipality Name3,Miles\rPlanned,NaN,NaN,NaN,NaN,NaN
1,2022,0232,SECOND STREET P,Bustleton Pk / SR 2065,0094,0000,Swamp Rd / SR 2036,0150,1676.0,WRIGHTSTOWN,NORTHAMPTON,NaN,3.09
2,2022,0313,DUBLIN PK,Morgan Creek / Bridge,0082,0000,Bedminster Rd / Souderton Rd / SR 0113,0212,1500.0,RICHLAND,HILLTOWN,EAST ROCKHILL,6.40
3,2022,0313,SWAMP RD,Bedminster Rd / Souderton Rd / SR 0113,0222,0000,Curly Hill Rd / SR 1008,0282,2345.0,DUBLIN,NEW BRITAIN,HILLTOWN,3.72
4,2022,0313,SWAMP RD,Curly Hill Rd / SR 1008,0292,0000,Doylestown / Plumstead Twp. Line,0342,948.0,NEW BRITAIN,DOYLESTOWN,NaN,2.09


In [13]:
column_names = ["Year", "SR", "Road Name", "From", "SegmentFrom", "OffsetFrom", "To", "SegmentTo", "OffsetTo", "Municipality1", "Municipality2", "Municipality3", "Miles Panned"]

In [164]:
len(df)

78

In [14]:
table = tabula.read_pdf(file, pages=1, pandas_options={'header': None})[0]
df = pd.DataFrame(table)

col_to_drop = [4, 8]
df.drop(col_to_drop, axis=1, inplace=True)

df.columns=column_names
df.head()




ValueError: Length mismatch: Expected axis has 11 elements, new values have 13 elements